In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import re
import gc
from sklearn.metrics import mean_squared_error, roc_auc_score

from xgboost import XGBClassifier, XGBRFRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from lightgbm import LGBMModel,LGBMClassifier
import copy

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-whdte3g7 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [13]:
from sklearn.feature_selection import RFE

# clf = SVC(kernel='sigmoid', random_state=42, probability=True, C=0.1, class_weight='balanced')
clf = LogisticRegression(solver='sag', random_state=42, class_weight='balanced')
# clf = XGBClassifier(
#         random_state = 42,
#         objective='binary:logistic',
#         n_estimators=500,
#         learning_rate=1e-3,
#         max_depth=3,
#         subsample=0.7,
#         scale_pos_weight=np.unique(y_train, return_counts=True)[1][0] / np.unique(y_train, return_counts=True)[1][1]
#     )
    

val_evals = []
aucs = []
f1s = []
for fidx in range(5):
    X_train_ = np.load('/work/zg78/som_folds/X_train_{}.npy'.format(fidx))
    y_train_raw = np.load('/work/zg78/som_folds/cog_train_{}.npy'.format(fidx))
    X_val_ = np.load('/work/zg78/som_folds/X_val_{}.npy'.format(fidx))
    y_val_raw = np.load('/work/zg78/som_folds/cog_val_{}.npy'.format(fidx))
    covar_train = np.load('/work/zg78/som_folds/covar_train_{}.npy'.format(fidx))
    covar_val = np.load('/work/zg78/som_folds/covar_val_{}.npy'.format(fidx))
    
    X_train_raw = np.concatenate((X_train_, covar_train), axis=1)
    X_val_raw = np.concatenate((X_val_, covar_val), axis=1)
    
    
    X_train = X_train_raw[~np.isnan(X_train_raw).any(axis=1)]
    y_train = y_train_raw[~np.isnan(X_train_raw).any(axis=1)]
    X_val = X_val_raw[~np.isnan(X_val_raw).any(axis=1)]
    y_val = y_val_raw[~np.isnan(X_val_raw).any(axis=1)]
    
    print(X_train.shape, y_train.shape, np.unique(y_val, return_counts=True))
    
    std_scaler = StandardScaler()
    std_scaler.fit(X_train)
    X_train = std_scaler.transform(X_train)
    X_val = std_scaler.transform(X_val)
        
    clf.fit(X_train, y_train)
    
    train_acc = metrics.accuracy_score(y_train, clf.predict(X_train))
    val_acc = metrics.accuracy_score(y_val, clf.predict(X_val))
    val_proba = clf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    f1 = metrics.f1_score(y_val, clf.predict(X_val))
    val_evals.append(val_acc)
    aucs.append(auc)
    f1s.append(f1)
    print(train_acc, val_acc, auc, f1, '\n')
    
print(sum(val_evals) / len(val_evals))
print(sum(aucs) / len(aucs))
print(sum(f1s) / len(f1s))
    
    

(52, 20) (52,) (array([0, 1]), array([9, 4]))
0.8846153846153846 0.6923076923076923 0.8055555555555556 0.5 

(55, 20) (55,) (array([0, 1]), array([8, 2]))
0.8909090909090909 0.7 0.5 0.4 

(50, 20) (50,) (array([0, 1]), array([14,  1]))
0.82 0.7333333333333333 0.5 0.0 

(52, 20) (52,) (array([0, 1]), array([11,  2]))
0.9423076923076923 0.5384615384615384 0.6818181818181819 0.25 

(51, 20) (51,) (array([0, 1]), array([10,  4]))
0.9607843137254902 0.5 0.5 0.0 

0.6328205128205128
0.5974747474747475
0.22999999999999998


In [14]:
from sklearn.feature_selection import RFE

# clf = SVC(kernel='sigmoid', random_state=42, probability=True, C=0.1, class_weight='balanced')
clf = LogisticRegression(solver='sag', random_state=42, class_weight='balanced')
# clf = XGBClassifier(
#         random_state = 42,
#         objective='binary:logistic',
#         n_estimators=500,
#         learning_rate=1e-3,
#         max_depth=3,
#         subsample=0.7,
#         scale_pos_weight=np.unique(y_train, return_counts=True)[1][0] / np.unique(y_train, return_counts=True)[1][1]
#     )
    

val_evals = []
aucs = []
f1s = []

for fidx in range(5):
    X_train_ = np.load('/work/zg78/som_folds/X_train_{}.npy'.format(fidx))
    y_train_raw = np.load('/work/zg78/som_folds/cog_train_{}.npy'.format(fidx))
    X_val_ = np.load('/work/zg78/som_folds/X_val_{}.npy'.format(fidx))
    y_val_raw = np.load('/work/zg78/som_folds/cog_val_{}.npy'.format(fidx))
    covar_train = np.load('/work/zg78/som_folds/covar_train_{}.npy'.format(fidx))
    covar_val = np.load('/work/zg78/som_folds/covar_val_{}.npy'.format(fidx))
    
    X_train_raw = np.concatenate((X_train_, covar_train), axis=1)
    X_val_raw = np.concatenate((X_val_, covar_val), axis=1)
    
    
    X_train = X_train_raw[~np.isnan(X_train_raw).any(axis=1)]
    y_train = y_train_raw[~np.isnan(X_train_raw).any(axis=1)]
    X_val = X_val_raw[~np.isnan(X_val_raw).any(axis=1)]
    y_val = y_val_raw[~np.isnan(X_val_raw).any(axis=1)]
        
    std_scaler = StandardScaler()
    std_scaler.fit(X_train)
    X_train = std_scaler.transform(X_train)
    X_val = std_scaler.transform(X_val)
    selector = RFE(clf, n_features_to_select=8, step=1)
#     clf.fit(X_train, y_train)
    selector = selector.fit(X_train, y_train)

    train_acc = metrics.accuracy_score(y_train, selector.predict(X_train))
    val_acc = metrics.accuracy_score(y_val, selector.predict(X_val))
    val_proba = selector.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    f1 = metrics.f1_score(y_val, selector.predict(X_val))
    val_evals.append(val_acc)
    aucs.append(auc)
    f1s.append(f1)

    print(train_acc, val_acc, auc, f1, '\n')
    
print(sum(val_evals) / len(val_evals))
print(sum(aucs) / len(aucs))
print(sum(f1s) / len(f1s))

0.7884615384615384 0.7692307692307693 0.7777777777777778 0.5714285714285715 

0.8 0.6 0.5 0.3333333333333333 

0.8 0.8 0.5 0.0 

0.9038461538461539 0.5384615384615384 0.7272727272727273 0.25 

0.9019607843137255 0.5714285714285714 0.375 0.0 

0.6558241758241758
0.5760101010101011
0.23095238095238096
